# Исследование надежности заемщиков

## Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Описание данных

- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gende`r — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

## Открытие файла и изучение данных

Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраним переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [4]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

Выведем первые 20 строчек датафрейма `data` на экран.**

In [5]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем основную информацию о датафрейме с помощью метода `info()`

data.info()

## Предобработка данных

### Удаление пропусков

**Выведем количество пропущенных значений для каждого столбца**

In [11]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом**

In [14]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()**

In [15]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**

In [16]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится вам для исследования.

**Выведем перечень уникальных значений столбца `children`**

In [18]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [19]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [20]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)


**Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [21]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.**

In [22]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [24]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов


**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверим остальные столбцы.**

In [25]:
data['education'] = data['education'].str.lower()

**Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [26]:
data.duplicated().sum()

71

In [27]:
data = data.drop_duplicates()

### Категоризация данных

**На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`.**

In [28]:
def categorize_income(income):
    
    try:
        if 0 <= income <= 30000:
            return 'E - до 30 000'
        elif 30001 <= income <= 50000:
            return 'D - до 50 000'
        elif 50001 <= income <= 200000:
            return 'C - до 200 000'
        elif 200001 <= income <= 1000000:
            return 'B - до 1 000 000'
        elif income >= 1000001:
            return 'A - свыше 1 000 000'
    except:
        pass

In [29]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [32]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

In [33]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [34]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследуем данные и ответим на вопросы

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [40]:
def pivot_data(category):
    
     #функция принимает в качестве аргумента столбец с признаком,по которому будет происходить расчет зависимости
    #и возвращает сводную таблицу для наглядности, с указанием общего числа кредитополучателей, должников и доли числа должников 
    #от общего количества клиентов в процентах
    
    
    pivot_data = data.pivot_table(index = category, values = 'debt', aggfunc = ['count','sum','mean',])
    pivot_data.columns = ['Всего кредитополучателей', 'Всего должников', 'Доля должников']
    pivot_data['Доля должников'] = pivot_data['Доля должников'] * 100
    return pivot_data

display(pivot_data(data['children']).sort_values(by='Доля должников', ascending=False))

,Всего кредитополучателей,Всего должников,Доля должников
children,,,
4,41,4,9.756098
2,2052,194,9.454191
1,4808,444,9.234609
3,330,27,8.181818
0,14091,1063,7.543822
5,9,0,0.000000


**Вывод:** Количество детей не влияет на возврат кредита в срок. Влияет их наличие или отсутствие.

Исходя из данных, можно сделать вывод, что скорее влияет само наличие детей, нежели их количество: долников, не имеющих детей, почти на 2 процента меньше, чем всех остальных (исключение: люди с тремя детьми. Но тут, возможно, дело в том, что таких людей, в принципе, меньше - 330 человек значительно меньше 14091, что бы сравнивать эти две категории)


#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [42]:
display(pivot_data(data['family_status']).sort_values(by='Доля должников', ascending=False))

,Всего кредитополучателей,Всего должников,Доля должников
family_status,,,
Не женат / не замужем,2796,273,9.763948
гражданский брак,4134,385,9.313014
женат / замужем,12261,927,7.560558
в разводе,1189,84,7.064760
вдовец / вдова,951,63,6.624606


**Вывод:**
Семейное положение влияет на возврат кредита

Тут мы можем заметить следующую закономерность: люди, заключившие официальный брак, с бОльшей охотой вохвращают кредиты. Связанно это может быть с тем, что и брак, и кредит - ответственность и обязательства, и люди, заключающие официальный брак, более ответственные 

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?


In [43]:
display(pivot_data(data['total_income_category']).sort_values(by='Доля должников', ascending=False))

,Всего кредитополучателей,Всего должников,Доля должников
total_income_category,,,
E - до 30 000,22,2,9.090909
C - до 200 000,15921,1353,8.498210
A - свыше 1 000 000,25,2,8.000000
B - до 1 000 000,5014,354,7.060231
D - до 50 000,349,21,6.017192


**Вывод:** Уровень дохода может влиять на возврат кредита.

Тут сложно ответить точно, т.к. изначально данные неточные - более чем в 2000 поцизий не был указан уровень дохода. Данные пропуски были заменены медианными значениями, что не может отображать истинную картину. 
Но из имеющихся данных можно сделать вывод, что чем выше доход, тем выше вероятность возврата.

#### Как разные цели кредита влияют на его возврат в срок?

In [44]:
display(pivot_data(data['purpose_category']).sort_values(by='Доля должников', ascending=False))

,Всего кредитополучателей,Всего должников,Доля должников
purpose_category,,,
операции с автомобилем,4279,400,9.347978
получение образования,3988,369,9.252758
проведение свадьбы,2313,183,7.911803
операции с недвижимостью,10751,780,7.255139


**Вывод:** цель кредита влияет на возврат

Здесь все прозаично: не заплатил за кредит - негде будет жить. Потому люди более ответственно относятся к выплате кредита за недвижимость.
Проведение свадьбы - чуть выше мы уже поняли, что официальны брак - показатель ответственности человека.
Образование, к сожалению, не гарантирует высокий и своевременный доход. Плюс, подобные кредиты, как правило, берут моложые люди - не очень платежеспособные и ответственные.
Автомобиль - это риск и расходы, тут вопрос не столько в ответственности, сколько в случае.

## Общий вывод.

Оценивать платежеспособность по какому-то одному признаку некорректно. 

К примеру, человек, имеющий 2 детей (самый большой процент невозврата по признаку количества детей) может зарабатывать до 1 000 000 (самый низкий процент невозврата по признаку уровня дохода). Потому корректнее было бы искать зависимости в связке нескольких признаков.